# <h1 id="problema">Contexto analítico</h1>

Este caso se basa en un reto propuesto por Red Hat en la plataforma Kaggle. (https://www.kaggle.com/c/predicting-red-hat-business-value/data) 

Red Hat es capaz de obtener una importante cantidad de información de comportamiento de sus clientes. El interes de la compañia se basa en la busqueda de mejores metodos para analizar sus clientes para predecir sobre cuales deberian hacer un especial enfasis que podrian convertirse en clientes de alto valor.

En este reto se propone crear un algoritmo de clasificación que identifique los clientes con mayor potencial para convertirse en clientes de alto valor para Red Hat basados en su actividad.

Se cuenta con dos datasets (people.csv y act_train.csv) los cuales cargaremos directamente a Colab.


In [ ]:
from IPython.display import display

import re
import numpy as np
import pandas as pd

pd.options.display.float_format = '{:.3f}'.format

In [ ]:
path = '../data/{}.parquet'

df_activity = pd.read_parquet(path.format('act_train'))
df_people = pd.read_parquet(path.format('people'))

df_activity.set_index('people_id', drop=True, inplace=True)
df_people.set_index('people_id', drop=True, inplace=True)

display(df_people.head(2))
display(df_activity.head(2))


In [ ]:
# Merge both df by index (people_id)
# df = df_activity.join(df_people, lsuffix='_activity')
df = pd.merge(df_people, df_activity, on='people_id', suffixes=('', '_activity'))

print(f'Before:')
print(f'   Activity: {df_activity.shape}')
print(f'   People:    {df_people.shape} \n')
print(f'After:')
print(f'   New:      {df.shape}')

In [ ]:
empty_percentage = (df.isna().sum()/df.shape[0])
empty_percentage = empty_percentage[empty_percentage > 0]
display(empty_percentage)

# Drop columns with most NAs
columns_more_empty = empty_percentage[empty_percentage >= 0.5].index
df.drop(columns=columns_more_empty, inplace=True)



In [ ]:
# Replace empty values by the most common
df['char_10_activity'].fillna(df['char_10_activity'].mode()[0], inplace=True)

empty_percentage = (df.isna().sum()/df.shape[0])
empty_percentage = empty_percentage[empty_percentage > 0]
display(empty_percentage)

In [ ]:
display(df.dtypes.value_counts())

In [ ]:
# Replace bool types for number types
df.replace({True: 1, False: 0}, inplace=True)
display(df.dtypes.value_counts())

In [ ]:
# Get different times formats to int type from date columns 
def split_date(dataframe, column='date', sufix='', times=('day', 'weekday', 'week', 'month', 'year', 'quarter')):
    series_time = pd.to_datetime(dataframe[column])
    dataframe.drop(columns=[column], inplace=True)

    for time in times:
        dataframe[time + sufix] = getattr(series_time.dt, time)


split_date(df)
split_date(df, 'date_activity', '_activity')

display(df.dtypes.value_counts())

In [ ]:
print('UNIQUE VALUES\n')
display(df.select_dtypes('object').nunique())

df.select_dtypes('object').head(2)

In [ ]:
# Deleting text from some columns to convert to int
columns_del_txt = ['activity_id', 'char_10_activity', 'group_1']
df[columns_del_txt] = df[columns_del_txt].applymap(
    lambda x: re.findall(r'(\d+$)', x)[0]).astype('int64', copy=False)

display(df.dtypes.value_counts())
display(df[columns_del_txt].head(3))

print('UNIQUE VALUES\n')
df.select_dtypes('object').nunique()

In [ ]:
categorical = df.select_dtypes('object')
df_one_hot_enc = pd.get_dummies(categorical)

df.drop(columns=categorical.columns, inplace=True)
df = pd.concat([df, df_one_hot_enc], axis=1)

df.dtypes.value_counts()

In [ ]:
columns_range = np.linspace(0, len(df.columns), int(len(df.columns)/10), dtype=int)

for index in range(len(columns_range)):
    try:
        start = columns_range[index]
        end = columns_range[index + 1]
        display(df.iloc[0:3, start:end])
    except IndexError:
        pass

In [ ]:
df.to_parquet('../data/redhat_v1.parquet')